In [346]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [347]:
benchmark_scores = pd.read_csv('../data/processed/benchmark_scores_2021_2023_cleaned.csv')
df = benchmark_scores.copy()

In [348]:
df.head()

,year,university_code,program_type,uni_major_code,major_code,major_name,subject_group_code,admission_method,score_scale,benchmark_score
0,2021,FTU2,DT,NTS01,7310101,Kinh tế,D01,Tốt nghiệp THPT,30,28.55
1,2021,GDU,DT,7380101,73801,Luật,A01,Tốt nghiệp THPT,30,15.00
2,2021,GDU,DT,7380101,73801,Luật,XDHB,Xét học bạ,30,16.50
3,2021,GDU,DT,7480201,74802,Công nghệ thông tin,A01,Tốt nghiệp THPT,30,16.00
4,2021,GDU,DT,7480201,74802,Công nghệ thông tin,A01,Tốt nghiệp THPT,30,16.50


##### Các feature cho mô hình:
+   `year`
+   `university_code`
+   `program_type`
+   `major_code`
+   `subject_group_code`
+   `admission_method`   
##### Label:
+   `benchmark_score`

---

In [349]:
df.duplicated().sum()

np.int64(0)

In [350]:
df.isnull().sum()

year                  0
university_code       0
program_type          0
uni_major_code        0
major_code            0
major_name            0
subject_group_code    0
admission_method      0
score_scale           0
benchmark_score       0
dtype: int64

In [351]:
df

,year,university_code,program_type,uni_major_code,major_code,major_name,subject_group_code,admission_method,score_scale,benchmark_score
0,2021,FTU2,DT,NTS01,7310101,Kinh tế,D01,Tốt nghiệp THPT,30,28.55
1,2021,GDU,DT,7380101,73801,Luật,A01,Tốt nghiệp THPT,30,15.00
2,2021,GDU,DT,7380101,73801,Luật,XDHB,Xét học bạ,30,16.50
3,2021,GDU,DT,7480201,74802,Công nghệ thông tin,A01,Tốt nghiệp THPT,30,16.00
4,2021,GDU,DT,7480201,74802,Công nghệ thông tin,A01,Tốt nghiệp THPT,30,16.50
...,...,...,...,...,...,...,...,...,...,...
7707,2023,VNUHCM-USSH,DT,7760101,7760101,Công tác xã hội,C00,Tốt nghiệp THPT,30,24.70
7708,2023,VNUHCM-USSH,DT,7760101,7760101,Công tác xã hội,DGNL,Đánh giá năng lực ĐHQG-HCM,1200,670.00
7709,2023,VNUHCM-USSH,DT,7810103,7810103,Quản trị dịch vụ du lịch và lữ hành,D01,Tốt nghiệp THPT,30,25.80
7710,2023,VNUHCM-USSH,DT,7810103,7810103,Quản trị dịch vụ du lịch và lữ hành,C00,Tốt nghiệp THPT,30,27.40


- Chuyển thang điểm 1200 sang 30 (tránh tồn tại các giá trị ngoại lai)
- Xoá các mẫu trùng lập

In [352]:
# Chuyển đổi các thang điểm về lại thành thang điểm 30
df.loc[df['score_scale'] == 1200,'benchmark_score'] = df.loc[df['score_scale'] == 1200,'benchmark_score'] * 30.0 / 1200.0

df.drop(columns=['score_scale','major_name','uni_major_code'],inplace=True)
print(df.duplicated().sum())

# drop_duplicated
df.drop_duplicates(subset=['year','university_code','program_type','subject_group_code','major_code','admission_method'],\
                     keep='first',inplace=True)

302


In [353]:
def group_top_n_labels(series, top_n=10):
    '''Giữ lại n dữ liệu xuất hiện nhiều nhất, còn lại sẽ được chuyển thành Other'''
    top_values = series.value_counts().nlargest(top_n).index
    return series.apply(lambda x: x if x in top_values else 'Other')

In [354]:
df['university_code'] = group_top_n_labels(df['university_code'],20)
df['major_code'] = group_top_n_labels(df['major_code'],40)
df['subject_group_code'] = group_top_n_labels(df['subject_group_code'],20)

In [355]:
# Trường hợp 1: Duplicated cả x và y
df.drop_duplicates(inplace=True)

# Trường hợp 2: Duplicated x nhưng lại khác y
cols_X = ['year', 'university_code', 'major_code', 'subject_group_code', 'program_type', 'admission_method']
df.drop_duplicates(subset=cols_X, keep='first', inplace=True)

print(df.duplicated(subset=['year','university_code','major_code','subject_group_code','program_type','admission_method']).sum())
print(df.duplicated().sum())

0
0


split DataFrame thành x và y

In [356]:
x = df.drop(columns='benchmark_score').reset_index(drop=True)
y = df.loc[:,'benchmark_score'].reset_index(drop=True)

In [357]:
x

,year,university_code,program_type,major_code,subject_group_code,admission_method
0,2021,Other,DT,7310101,D01,Tốt nghiệp THPT
1,2021,GDU,DT,73801,A01,Tốt nghiệp THPT
2,2021,GDU,DT,73801,XDHB,Xét học bạ
3,2021,GDU,DT,74802,A01,Tốt nghiệp THPT
4,2021,GDU,DT,74802,XDHB,Xét học bạ
...,...,...,...,...,...,...
3412,2023,VNUHCM-USSH,DT,7760101,C00,Tốt nghiệp THPT
3413,2023,VNUHCM-USSH,DT,7760101,DGNL,Đánh giá năng lực ĐHQG-HCM
3414,2023,VNUHCM-USSH,DT,7810103,D01,Tốt nghiệp THPT
3415,2023,VNUHCM-USSH,DT,7810103,C00,Tốt nghiệp THPT


In [358]:
df[['year','university_code','program_type','major_code','subject_group_code','admission_method']] = \
    df[['year','university_code','program_type','major_code','subject_group_code','admission_method']].astype('category')

In [359]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Encode categorical attributes in x
ct = ColumnTransformer(transformers=[
    ('encoder', OneHotEncoder(), ['university_code', 'program_type', 'major_code', 'subject_group_code', 'admission_method'])
], remainder='passthrough')
x_encoded = ct.fit_transform(x)

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'year' column
label_encoder = LabelEncoder()
x['year'] = label_encoder.fit_transform(x['year'])

In [ ]:
# df.to_csv('data_modeling.csv',index=False)